In [1]:
1+1

2

In [3]:
import os
import logging
from typing import List, TypedDict, Literal

from dotenv import load_dotenv
from langchain.agents import Tool
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import create_react_agent

# ==============================================================================
# 0. 로깅 및 설정 클래스
# ==============================================================================

# 애플리케이션 전반에 걸쳐 일관된 로깅을 설정합니다.
# 이를 통해 디버깅 시 특정 모듈의 로그만 선택적으로 볼 수 있습니다.
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# LangChain과 LangGraph의 상세한 내부 동작을 확인하고 싶을 때 아래 주석을 해제합니다.
# logging.getLogger("langchain").setLevel(logging.DEBUG)
# logging.getLogger("langgraph").setLevel(logging.DEBUG)


class Config:
    """
    애플리케이션 설정을 관리하는 클래스입니다.
    환경 변수 로드 및 주요 파라미터를 중앙에서 관리합니다.
    """

    def __init__(self):
        load_dotenv()
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if not self.openai_api_key:
            raise ValueError("OPENAI_API_KEY 환경 변수를 설정해야 합니다.")

        self.llm_model = "gpt-4o-mini"
        self.temperature = 0
        self.max_iterations = 5

    @property
    def recursion_limit(self) -> int:
        """에이전트의 최대 재귀 호출 횟수를 계산합니다."""
        return self.max_iterations * 2 + 1


# ==============================================================================
# 1. 도구 정의 클래스
# ==============================================================================


class ToolKit:
    """
    에이전트가 사용할 도구들을 정의하고 관리하는 클래스입니다.
    도구를 추가하거나 수정할 때 이 클래스만 변경하면 됩니다.
    """

    def __init__(self):
        self._tools = self._create_tools()

    def get_tools(self) -> List[Tool]:
        """정의된 도구 리스트를 반환합니다."""
        return self._tools

    def _create_tools(self) -> List[Tool]:
        """실제 도구들을 생성합니다."""
        population_tool = Tool(
            name="PopulationLookup",
            func=self.population_lookup,
            description="대한민국 주요 도시의 인구를 조회합니다. 도시 이름을 인수로 받습니다. (예: 서울, 부산)",
        )
        add_two_tool = Tool(
            name="AddTwo", func=self.add_two, description="숫자 입력 시 2를 더합니다."
        )
        return [population_tool, add_two_tool]

    @staticmethod
    def population_lookup(city: str) -> str:
        """도시 이름을 받아 인구를 조회합니다."""
        data = {
            "서울": "9,515,000명 (2023년 기준)",
            "부산": "3,343,000명 (2023년 기준)",
        }
        logger.info(f"인구 조회 도구 실행: city='{city}'")
        if city in data:
            return data[city]
        else:
            # 의도적으로 에러를 발생시켜 fallback 로직을 테스트합니다.
            raise ValueError(
                f"'{city}'의 인구 정보를 찾을 수 없습니다. 도구는 서울, 부산만 조회 가능합니다."
            )

    @staticmethod
    def add_two(x: str) -> str:
        """입력된 숫자에 2를 더합니다."""
        try:
            return str(float(x) + 2)
        except Exception:
            return "도구 실행 실패: 숫자만 입력해 주세요."


# ==============================================================================
# 2. 에이전트 그래프 클래스
# ==============================================================================


class AgentState(TypedDict):
    """그래프의 상태를 정의합니다."""

    messages: List[BaseMessage]


# Literal 타입을 사용하여 라우터가 반환할 수 있는 값을 명시적으로 제한합니다.
RouterChoice = Literal["fallback", "end", "continue"]


class ErrorHandlerAgent:
    """
    에러 처리 로직이 포함된 ReAct 에이전트 워크플로우를 캡슐화한 클래스입니다.
    """

    def __init__(self, config: Config, tools: List[Tool]):
        self.config = config
        self.llm = ChatOpenAI(
            model=self.config.llm_model,
            temperature=self.config.temperature,
            verbose=False,  # 클래스 레벨에서 로깅을 제어하므로 False로 설정
        )
        self.tools = tools
        self.graph = self._build_graph()

    def _build_graph(self) -> StateGraph:
        """LangGraph 워크플로우를 구성하고 컴파일합니다."""
        graph = StateGraph(AgentState)

        react_agent = create_react_agent(self.llm, self.tools)

        graph.add_node("agent", react_agent)
        graph.add_node("fallback", self._fallback_node)

        graph.set_entry_point("agent")

        graph.add_conditional_edges(
            "agent",
            self._should_fallback,
            {
                "fallback": "fallback",
                "end": END,
                "continue": "agent",
            },
        )
        graph.add_edge("fallback", END)

        logger.info("에이전트 그래프 구성 완료")
        return graph.compile()

    def _should_fallback(self, state: AgentState) -> RouterChoice:
        """
        에이전트의 상태를 보고 다음 행동(fallback, 종료, 계속)을 결정하는 라우터입니다.
        """
        logger.info("--- [분기 확인] 다음 경로를 결정합니다. ---")
        tool_error_count = sum(
            1
            for msg in state["messages"]
            if isinstance(msg, ToolMessage) and msg.content.startswith("Error:")
        )

        logger.info(f"현재까지 누적된 도구 오류 횟수: {tool_error_count}")

        if tool_error_count >= 1:
            logger.warning("오류 횟수 임계값 도달. Fallback 경로로 이동합니다.")
            return "fallback"

        last_message = state["messages"][-1]
        if isinstance(last_message, AIMessage) and not last_message.tool_calls:
            logger.info("에이전트가 최종 답변을 생성했습니다. 그래프를 종료합니다.")
            return "end"

        logger.info("에이전트가 작업을 계속 진행합니다.")
        return "continue"

    def _fallback_node(self, state: AgentState) -> AgentState:
        """도구 사용 실패 시 순수 LLM이 대신 응답을 생성하는 노드입니다."""
        logger.info("--- FALLBACK TO LLM ---")
        history = state["messages"]
        response = self.llm.invoke(history)
        return {"messages": history + [response]}

    def invoke(self, query: str) -> dict:
        """에이전트를 실행하고 최종 결과를 반환합니다."""
        inputs = {"messages": [HumanMessage(content=query)]}
        return self.graph.invoke(
            inputs, {"recursion_limit": self.config.recursion_limit}
        )


# ==============================================================================
# 3. 메인 실행 블록
# ==============================================================================


def main():
    """
    애플리케이션의 전체 실행 흐름을 제어하는 메인 함수입니다.
    """
    try:
        # 1. 설정 및 도구 초기화
        config = Config()
        toolkit = ToolKit()
        tools = toolkit.get_tools()

        # 2. 에이전트 생성
        agent = ErrorHandlerAgent(config, tools)

        # 3. 에이전트 실행
        query = "대구의 인구는 몇 명이야?"
        logger.info(f"--- 에이전트 실행 시작 (Query: '{query}') ---")
        response = agent.invoke(query)

        # 4. 결과 출력
        print("\n\n--- 최종 대화 기록 ---")
        if response and "messages" in response:
            for m in response["messages"]:
                print(f"[{m.__class__.__name__}]: {getattr(m, 'content', repr(m))}")
        else:
            logger.error("최종 응답을 가져오지 못했습니다.")

    except Exception as e:
        logger.error(f"애플리케이션 실행 중 오류 발생: {e}", exc_info=True)


if __name__ == "__main__":
    main()

2025-07-05 00:10:59,792 - INFO - 에이전트 그래프 구성 완료
2025-07-05 00:10:59,794 - INFO - --- 에이전트 실행 시작 (Query: '대구의 인구는 몇 명이야?') ---
2025-07-05 00:11:00,832 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-05 00:11:00,882 - INFO - 인구 조회 도구 실행: city='대구'
2025-07-05 00:11:02,079 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-05 00:11:02,095 - INFO - --- [분기 확인] 다음 경로를 결정합니다. ---
2025-07-05 00:11:02,097 - INFO - 현재까지 누적된 도구 오류 횟수: 1
2025-07-05 00:11:02,099 - WARNING - 오류 횟수 임계값 도달. Fallback 경로로 이동합니다.
2025-07-05 00:11:02,101 - INFO - --- FALLBACK TO LLM ---
2025-07-05 00:11:04,199 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




--- 최종 대화 기록 ---
[HumanMessage]: 대구의 인구는 몇 명이야?
[AIMessage]: 
[ToolMessage]: Error: ValueError("'대구'의 인구 정보를 찾을 수 없습니다. 도구는 서울, 부산만 조회 가능합니다.")
 Please fix your mistakes.
[AIMessage]: 현재 이 도구는 서울과 부산의 인구 정보만 조회할 수 있습니다. 다른 도시의 인구 정보는 제공할 수 없습니다. 서울이나 부산의 인구를 알고 싶으신가요?
[AIMessage]: 대구의 인구는 약 2.4백만 명(240만 명) 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으니, 최신 정보를 확인하는 것이 좋습니다.
